### What does this code do?

Compute realized variances, covariances, and semi-variances from the hourly prices for each country and market pair.



In [1]:
# 02_compute_realized_volatility.py

import pandas as pd
import numpy as np
from itertools import combinations
import os



In [2]:
# Load the parquet dataset
df = pd.read_parquet("filtered_data.parquet")
df.head()

,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),Intraday Period (CET/CEST),Intraday Price (EUR/MWh),Start DateTime,range,TimeDiff,NonHourly
0,01/01/2021 00:00:00 - 01/01/2021 01:00:00,BZN|ES,Without sequence,50.87,NaN,NaN,2021-01-01 00:00:00,False,NaT,False
1,01/01/2021 01:00:00 - 01/01/2021 02:00:00,BZN|ES,Without sequence,48.19,NaN,NaN,2021-01-01 01:00:00,False,0 days 01:00:00,False
2,01/01/2021 02:00:00 - 01/01/2021 03:00:00,BZN|ES,Without sequence,44.68,NaN,NaN,2021-01-01 02:00:00,False,0 days 01:00:00,False
3,01/01/2021 03:00:00 - 01/01/2021 04:00:00,BZN|ES,Without sequence,38.50,NaN,NaN,2021-01-01 03:00:00,False,0 days 01:00:00,False
4,01/01/2021 04:00:00 - 01/01/2021 05:00:00,BZN|ES,Without sequence,36.80,NaN,NaN,2021-01-01 04:00:00,False,0 days 01:00:00,False


### Disambiguate DST duplicates using a counter

In [4]:
# Step 1: Sort and identify duplicates in Start DateTime + Area
df = df.sort_values(['Area', 'Start DateTime'])

# Step 2: Mark duplicated datetimes within each Area
df['dup_rank'] = df.groupby(['Area', 'Start DateTime']).cumcount()

# Step 3: Add a slight offset (e.g. 1 second) to duplicated timestamps
df['Start DateTime Unique'] = df['Start DateTime'] + pd.to_timedelta(df['dup_rank'], unit='s')

# Step 4: Pivot using the unique datetime
df_pivot = df.pivot(index='Start DateTime Unique', columns='Area', values='Day-ahead Price (EUR/MWh)')
df_pivot = df_pivot.sort_index()


In [6]:
df = df.drop_duplicates(subset=['Start DateTime', 'Area'])
df_pivot = df.pivot(index='Start DateTime', columns='Area', values='Day-ahead Price (EUR/MWh)')

df_pivot.head()


Area,BZN|ES,BZN|FR,BZN|PT
Start DateTime,,,
2021-01-01 00:00:00,50.87,50.87,50.87
2021-01-01 01:00:00,48.19,48.19,48.19
2021-01-01 02:00:00,44.68,44.68,44.68
2021-01-01 03:00:00,38.50,42.92,38.50
2021-01-01 04:00:00,36.80,40.39,36.80


In [7]:
# Compute hourly log returns
log_returns = np.log(df_pivot).diff().dropna()

# Compute realized variances
realized_variance = log_returns**2


realized_variance.head()


C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Area,BZN|ES,BZN|FR,BZN|PT
Start DateTime,,,
2021-01-01 01:00:00,0.002929,0.002929,0.002929
2021-01-01 02:00:00,0.005719,0.005719,0.005719
2021-01-01 03:00:00,0.022162,0.001615,0.022162
2021-01-01 04:00:00,0.002039,0.003691,0.002039
2021-01-01 05:00:00,0.000046,0.000022,0.000046


In [8]:
# Compute realized covariances
realized_covariance = {
    f'{i}_{j}': log_returns[i] * log_returns[j]
    for i, j in combinations(log_returns.columns, 2)
}

# Convert covariances to DataFrame
realized_cov_df = pd.DataFrame(realized_covariance, index=log_returns.index)


realized_cov_df.head()



,BZN|ES_BZN|FR,BZN|ES_BZN|PT,BZN|FR_BZN|PT
Start DateTime,,,
2021-01-01 01:00:00,0.002929,0.002929,0.002929
2021-01-01 02:00:00,0.005719,0.005719,0.005719
2021-01-01 03:00:00,0.005983,0.022162,0.005983
2021-01-01 04:00:00,0.002744,0.002039,0.002744
2021-01-01 05:00:00,0.000032,0.000046,0.000032


In [ ]:
# Save for later modeling steps
realized_variance.to_parquet("parquet_files/realized_variance.parquet")
realized_cov_df.to_parquet("parquet_files/realized_covariance.parquet")
log_returns.to_parquet("parquet_files/log_returns.parquet")